In [ ]:
# Syed Abdul Rafey Saleem
# SAL21533673
# Coursework 1 (Hash Function)
# 07/11/2023

In [ ]:
# Importing math library for the ceil function 
import math
# Importing openpyxl to access the data from the excel file
import openpyxl
from openpyxl import load_workbook
# Importing prettyTable library to print data in a table form
!pip install prettytable
from prettytable import PrettyTable

In [ ]:
# PrintTable function takes the RetailNames and the corresponding data with the decoded index as parameters
def PrintTable(retail, data, indices):
    t = PrettyTable()
    t.field_names = ['Retail Name', 'Address', 'PostCode', 'Size Ban']

    for i in indices:
        #if statement to make sure that the data with only address1 doesnot print 'None' in place of Address2
        if (data[i][1] == None):
            Address = data[i][0] + ", " + data[i][2]
        else:
            Address = data[i][0] + ", " + data[i][1] + ", " + data[i][2]
        Postcode = data[i][3]
        Ban = data[i][4]
        # Adds rows in the table 
        t.add_row([retail[i], Address, Postcode, Ban])
    # The function returns the table t
    return t

workbook = load_workbook(filename = "CourseworkData.xlsx")
sheet = workbook['geolytix_retailpoints_v25_20220']
# count stores the number of rows containing data
count = len([row for row in sheet if not all([cell.value == None for cell in row])])
# l is the length of the list we will be storing our data in.
# l is set 1.3 times the actual data to reduce the load factor
l = math.ceil(count*1.3)

# Initializing the Key; RetailKey and Values; Data
RetailKey = ['']*l
Data = ['']*l
index = 1

# Storing the values from the excel file in a list
for row in sheet.rows:
    if (index == count):
        break
    # Starting from the secong index to avoid adding the headings from the excel sheet
    index = index + 1
    # Data Values to store the combination of address, postcode and size_ban
    DataValues = []
    # Finds the value from the excel sheet and stores it in the variables for each row
    Rname = sheet[('B' + str(index))].value
    # Storing the value in uppercase to make it in sync
    Rname = Rname.upper()
    Add1 = sheet[('E' + str(index))].value
    Add2 = sheet[('F' + str(index))].value
    Town = sheet[('G' + str(index))].value
    Postcode = sheet[('I' + str(index))].value
    ban = sheet[('P' + str(index))].value
    
    # All the values other than the retail name added in DataValues list
    DataValues.append(Add1) 
    DataValues.append(Add2) 
    DataValues.append(Town) 
    DataValues.append(Postcode) 
    DataValues.append(ban) 

# ASSIGNING KEYS TO THE DATA    
    # Calculating Hash Key using combination of Retail name and postcode as Postcode makes the hash key unique
    key = 0
    Rvalue = 0
    Pvalue = 0
    if (Postcode == None):
        continue
    # Adding all the ascii values of the characters in the retail name to Rvalue
    for i in range(0,len(Rname)):
        Rvalue = Rvalue + ord(Rname[i])
    # Adding all the ascii values of the characters in the postcode to Pvalue
    for j in range(0,len(Postcode)):
        Pvalue = Pvalue + ord(Postcode[j])
    
    # The hash key is found using the following equation and stored in key
    key = (Rvalue * Pvalue) % l
    
    # Finding another index for the key in case the space is already taken
    find = False
    while (find == False):
        if (RetailKey[key] == ''):
            RetailKey[key] = Rname
            Data[key] = DataValues 
            find = True
        else:
            # Incrementing the key by 13 to prevent clustering of the values
            key = (key + 13) % l


# RETRIEVING THE VALUES
terminate = '1'
decoded_index = []
# Lists to store the array of inputs
Retail = []
Post = []
index = 0
# While loop is implemented to get multiple search values from the user.
while (terminate != '0'):
    decode = 0
    decode2 = 0
    # Inputs stored in a temperory variable
    temp = input("\nEnter the name of the Retail: ")
    temp = temp.upper()
    Retail.append(temp)
    
    temp = input("\nEnter the postcode: ")
    temp = temp.upper()
    Post.append(temp)
    
    # Using same calculations to decode the key from the retail name and postcode
    for i in range(0,len(Retail[index])):
        decode = decode + ord(Retail[index][i])
    decode = decode % l

    for j in range(0,len(Post[index])):
        decode2 = decode2 + ord(Post[index][j])
    decode2 = decode2 % l
    
    decode = (decode * decode2) % l
    decoded_index.append(decode)
     
    terminate = input("\nEnter 0 to terminate the Program \nPress any other character to search again\n")
    index = index + 1


ReturnIndex = []
NotFound = []
# Comparing all the decode index values with the inputs
for j in range(0, len(decoded_index)):    
    find = False
    attempt = 0
    i = decoded_index[j]
    # Loops until finds the value with the same increment by 13
    while ((find == False) and (attempt != l)):
        if ((RetailKey[i] == Retail[j]) and (Data[i][3] == Post[j])):
            find = True
            print('\n')
            ReturnIndex.append(i)
        else:
            i = (i + 13) % l
            attempt = attempt + 1
            if (attempt == l):
                # NotFound stores the the inputs that are not found
                NotFound.append((Retail[j] + " with Postcode " + Post[j] + " not found"))

# Prints all the postcodes not found
[print(values) for values in NotFound]
print()
# Calls the PrintTable function to tabulate the result
table = PrintTable(RetailKey, Data, ReturnIndex)
print(table)